# Example Analysis

In [17]:
import xarray as xr
import pandas as pd
import dask.dataframe as dd
import hvplot.pandas  # noqa
import hvplot.dask  # noqa

# Build Dask Cluster
1. Use gui interface to create a new cluster with ~10 workers
2. Use < > to insert an "import Client" statement. This is critical because it is how your script knows to use yoru cluster. 
3. Execute client cell
4. Execute your Dask Cell. Note that once you have a cluster running you do not need to re-import the client. 
5. When finished, always shut down your cluster. 

In [1]:
from dask.distributed import Client

client = Client("tcp://10.0.128.167:42113")
client

Client Scheduler: tcp://10.0.128.167:42113 Dashboard: /user/daxsoule/proxy/8787/status,Cluster Workers: 8 Cores: 8 Memory: 10.00 GB


In [5]:
from pylab import rcParams

In [10]:
60*24

1440

In [13]:
#!head /home/jovyan/data/botpt/RS03ECAL-MJ03E-06-BOTPTA302/deployment0001_RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample_20140904T000000-20141128T060000.100000.nc\#fillmisma_resampled.nc

In [18]:
ds =xr.open_dataset('/home/jovyan/data/botpt/RS03ECAL-MJ03E-06-BOTPTA302/deployment0001_RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample_20191012T060000-20191014T235959.950000.nc')
ds

<xarray.Dataset>
Dimensions:          (index: 3960)
Coordinates:
  * index            (index) datetime64[ns] 2019-10-12T06:00:00 ... 2019-10-14T23:59:00
Data variables:
    bottom_pressure  (index) float32 ...

In [19]:
ds = xr.open_mfdataset('/home/jovyan/data/botpt/RS03ECAL-MJ03E-06-BOTPTA302/*_resampled.nc', parallel=True
                       ,combine='by_coords').chunk(1440)
ds

<xarray.Dataset>
Dimensions:          (index: 1914792)
Coordinates:
  * index            (index) datetime64[ns] 2015-12-26T12:00:00 ... 2019-10-14T23:59:00
Data variables:
    bottom_pressure  (index) float32 dask.array<chunksize=(1440,), meta=np.ndarray>

In [20]:
df = ds.to_dask_dataframe()
df = df.rename(columns={"index": "time"});
df = df.set_index('time')

In [21]:
df.head()

,bottom_pressure
time,
2015-12-26 12:00:00,2242.302734
2015-12-26 12:01:00,2242.293945
2015-12-26 12:02:00,2242.281982
2015-12-26 12:03:00,2242.275391
2015-12-26 12:04:00,2242.264160


In [22]:
df.describe()

,bottom_pressure
npartitions=1,
,float64
,...


In [ ]:
df.hvplot(y='bottom_pressure', datashade =True)